## Xview Dataset tiling
This is a component designed to tile the dataset provided by xview. 

In [ ]:
!pip install Pillow

In [ ]:

import os
import glob 
import logging
from PIL import Image


In [ ]:
def to_int(value):
    return int(value)

In [ ]:

#directory_path is the path to the folder with the unzipped .tif images from xview dataset 
directory_path = os.environ.get("directory_path")

#pattern is the search pattern a user can pass for the program to extract the files from 
pattern = os.environ.get("pattern")

#Each image is cropped using a rectangular window with edge_length tile_size_x and tile_size_y which has to be given in number of pixels 
tile_size_x = to_int(os.environ.get("tile_size_x"))
tile_size_y = to_int(os.environ.get("tile_size_y"))

#stride_x is the length in pixels the sliding window is moved to the right after each step
#stride_y is the length in pixels the sliding window is moved down after completing a row
#For tumbling window stride_x must equal tile_size_x and stride_y must equal tile_size_y
stride_x = to_int(os.environ.get("stride_x"))
stride_y = to_int(os.environ.get("stride_y"))

# log level
log_level = os.environ.get('log_level', 'INFO')


In [ ]:

def clear_directory(directory):
    logging.info(f'start clearing directory {directory}')
    for item in os.listdir(directory):
            item_path = os.path.join(directory, item)
            # Check if the item is a file
            if os.path.isfile(item_path):
                # If it's a file, remove it
                os.remove(item_path)
            # If it's a directory, recursively clear it
            elif os.path.isdir(item_path):
                clear_directory(item_path)
                os.rmdir(item_path)
    logging.ingo(f'directory {directory} cleared successfully')
    

In [ ]:

def sliding_window(directory_path, destination, tile_size_x, tile_size_y, stride_x, stride_y):
    
    if os.path.isdir(destination):
        logging.info(f'destination folder {destination} already exists')
        clear_directory(destination)
    else: 
        logging.info(f'destination folder {destination} does not exist yet, creating folder')
        os.makedirs(destination)
        logging.info(f'destination folder {destination} created successfully')

    items = glob.glob(pattern, recursive=True)
    
    for item in items:

        logging.info(f'grabbed item {item}, starting the tiling process')
        
        item_path = os.path.join(directory_path, item)
        clipped_item = os.path.join(destination, item)
        os.makedirs(clipped_item)
        
        image = Image.open(item_path)
        logging.info(f'{item} loaded successfully')
        width, height = image.size
        
        x_range = [0]
        while(x_range[-1] + stride_x + tile_size_x < width):
            x_range += [x_range[-1] + stride_x]
        
        y_range = [0]
        while(y_range[-1] + stride_y + tile_size_y < height):
            y_range += [y_range[-1] + stride_y]

        logging.info(f'successfully partitioned the height and width of {item}')
        
        counter = 0
        for x in x_range:
            for y in y_range:
                cropped = image.crop((x,y, x+tile_size_x, y+tile_size_y))

                logging.info(f'successfully extracted a tile from {item} at x={x} and y={y}')
                
                file_name = "{}".format(y)
                file_name += "_{}".format(x) + ".tif"
                path = os.path.join(clipped_item, file_name)
                
                cropped.save(path)

                logging.info(f'successfully saved tile from {item} at x={x} and y={y} in {path}')
                
        logging.info(f'Successfully finished tiling {item}')


In [ ]:
sliding_window(directory_path, destination, tile_size_x, tile_size_y, stride_x, stride_y)